# CityBikes

In [271]:
import pandas as pd
import json
from pprint import pprint

from pandas.core.interchange.dataframe_protocol import DataFrame

In [272]:
base_api = 'http://api.citybik.es/v2/networks'
data = pd.read_json(base_api)
networks = pd.json_normalize(data['networks'])
# networks.head()
networks[networks['location.city'].str.lower().str.contains('toronto')]

,id,name,href,company,gbfs_href,location.latitude,location.longitude,location.city,location.country,system,source,ebikes,license.name,license.url,scooters,instances
170,bixi-toronto,Bike Share Toronto,/v2/networks/bixi-toronto,"[Motivate International, Inc., PBSC Urban Solu...",https://tor.publicbikesystem.net/ube/gbfs/v1/,43.653226,-79.383184,"Toronto, ON",CA,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Send a request to CityBikes for the city of your choice. 

In [296]:
data = pd.read_json(base_api + '/bixi-toronto?fields=stations')
stations = data['network']['stations'].copy()
# stations

### Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes)

In [327]:
stations_list = []
for i, station in enumerate(stations):
    lat = f"{station['latitude']:.4f}"
    lon = f"{station['longitude']:.4f}"
    free = station['free_bikes']
    bikes = 'bike' if free == 1 else 'bikes'
    print(f"{i + 1}. ({lat}, {lon}) : {free} {bikes} available")
    # if lat == '43.6532': print(station['name'])
    stations_list.append(
        {
            'latitude': float(lat),
            'longitude': float(lon),
            'bikes_available': int(free)
        },
    )

1. (43.6859, -79.3763) : 1 bike available
2. (43.6678, -79.3081) : 9 bikes available
3. (43.6915, -79.2886) : 1 bike available
4. (43.6438, -79.3966) : 8 bikes available
5. (43.7607, -79.5000) : 0 bikes available
6. (43.6568, -79.3966) : 14 bikes available
7. (43.7741, -79.5005) : 0 bikes available
8. (43.6561, -79.3815) : 26 bikes available
9. (43.6588, -79.3696) : 7 bikes available
10. (43.6616, -79.4138) : 2 bikes available
11. (43.6634, -79.3921) : 18 bikes available
12. (43.6473, -79.3799) : 46 bikes available
13. (43.6397, -79.4487) : 16 bikes available
14. (43.6662, -79.3733) : 1 bike available
15. (43.6722, -79.3902) : 8 bikes available
16. (43.6902, -79.3501) : 1 bike available
17. (43.6713, -79.2945) : 10 bikes available
18. (43.6455, -79.4111) : 2 bikes available
19. (43.6617, -79.3378) : 8 bikes available
20. (43.6715, -79.4487) : 0 bikes available
21. (43.6731, -79.3187) : 10 bikes available
22. (43.7269, -79.2292) : 1 bike available
23. (43.7806, -79.1301) : 23 bikes avai

### Put your parsed results into a DataFrame.

In [323]:
stations_df = pd.DataFrame(station_list)
stations_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852 entries, 0 to 851
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   latitude         852 non-null    float64
 1   longitude        852 non-null    float64
 2   bikes_available  852 non-null    int64  
dtypes: float64(2), int64(1)
memory usage: 20.1 KB
